In [62]:
import pandas as pd
#from urllib.request import urlretrieve
#import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
#import nbformat
#sns.set_style('darkgrid')
#matplotlib.rcParams['font.size'] = 14
#matplotlib.rcParams['figure.figsize'] = (10, 6)
#matplotlib.rcParams['figure.facecolor'] = '#00000000'
import numpy as np
#from sklearn.preprocessing import StandardScaler
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
#%matplotlib inline

In [63]:
class CombinedDataset(Dataset):
    def __init__(self, dataset1, dataset2):
        self.dataset1 = dataset1
        self.dataset2 = dataset2

    def __len__(self):
        return min(len(self.dataset1), len(self.dataset2))

    def __getitem__(self, idx):
        sample1 = self.dataset1[idx]
        sample2 = self.dataset2[idx]
        return {'dim': sample1, 'target': sample2}

In [73]:

# Set the number of samples and the number of instances
num_samples = 100
num_instances = 1000

# Generate random values for x, y, z between 0 and 1
x = torch.rand(num_instances, num_samples)
y = torch.rand(num_instances, num_samples)
z = torch.rand(num_instances, num_samples)

# Generate random values for target between -1000 and 1000
target = torch.randint(-1000, 1001, (num_instances, num_samples))

# Combine x, y, z, and target into a single tensor for each instance
dim = torch.stack((x, y, z), dim=2)

dataset = CombinedDataset(dim, target)

val_size = num_instances//10
train_size = num_instances - val_size
print(val_size)
print(train_size)
train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)
print(f"Training set size: {len(train_ds)}")
print(f"Validation set size: {len(val_ds)}")
batch_size=20
train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=0, pin_memory=True)
#print(f"Training dataloadershape: {train_loader.shape}")

100
900
Training set size: 900
Validation set size: 100


In [74]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()
device

device(type='cpu')

In [75]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

for batch in train_loader:
    data1 = batch['dim']
    data2 = batch['target']
    print(data1.shape, data2.shape)
    data1 = to_device(data1, device)
    data2 = to_device(data2, device)
    break

torch.Size([20, 100, 3]) torch.Size([20, 100])


In [76]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [77]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
